In [ ]:
%pip install s3fs

# Load Libraries

In [1]:
from sagemaker.feature_store.feature_group import FeatureGroup

import boto3
import sagemaker
import pandas as pd
import time
import datetime
from time import  strftime, gmtime
from sagemaker.session import Session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Load Dataset
print('start time:', datetime.datetime.now())

s3_client = boto3.client("s3")
sagemaker_session = sagemaker.Session()
boto_session = boto3.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

sagemaker_client = boto_session.client(service_name='sagemaker',
                                       region_name=region)

featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime',
                                           region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment-rcf/notebook'
file_name = 'payment.csv'

data_location = 's3://{}/{}'.format(bucket_name, prefix_name)
data_file = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)

# Load data from csv
df_data = pd.read_csv(data_file)

print("---------------------------------")
print("Shape of dataframe :", df_data.shape)
print("data_location :", data_location)
print("data_file: ", data_file)
print("---------------------------------")
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df_data.head(5)

start time: 2024-05-04 23:28:11.912635
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
---------------------------------
Shape of dataframe : (51795, 25)
data_location : s3://eliezerraj-908671954593-dataset/payment-rcf/notebook
data_file:  s3://eliezerraj-908671954593-dataset/payment-rcf/notebook/payment.csv
---------------------------------


,id,fk_card_id,card_number,terminal_name,coord_x,coord_y,card_type,card_model,payment_at,mcc,amount,night_day,ic_night_day,wkend_wkday,ic_wkend_wkday,day_of_year,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_tx,time_btw_cc_tx,fraud
0,26965,843,111.111.000.843,TERM-221,222.0,261.0,CREDIT,VIRTUAL,2024-03-19 21:03:38.0,GAS,840.0,night,1,wkday,0,79.0,4,857.25,6,659.17,14,408.07,63,26,1
1,37036,636,111.111.000.636,TERM-59,38.0,34.0,CREDIT,VIRTUAL,2024-04-18 18:57:19.0,LAUNDRY,142.0,day,0,wkday,0,109.0,1,142.00,3,274.67,10,254.30,149,0,0
2,28164,538,111.111.000.538,TERM-24,77.0,45.0,CREDIT,CHIP,2024-03-23 15:47:16.0,SPORTING,408.0,day,0,wkend,1,83.0,3,209.33,3,209.33,6,207.17,8,1802,0
3,41128,517,111.111.000.517,TERM-97,8.0,85.0,CREDIT,CHIP,2024-04-30 18:52:27.0,CINEMA,106.0,day,0,wkday,0,121.0,2,109.50,2,109.50,8,210.75,22,997,0
4,37698,56,111.111.000.056,TERM-48,94.0,54.0,CREDIT,CHIP,2024-04-20 17:46:19.0,LAUNDRY,106.0,day,0,wkend,1,111.0,2,303.00,7,551.00,13,743.23,64,0,0


# Feature Engineiring

In [3]:
df_payment = df_data.filter(['fraud',
                            'payment_at',
                            'terminal_name',
                            'coord_x',
                            'coord_y',
                            'card_type',
                            'card_model',
                            'mcc',
                            'amount',
                            'tx_1d',
                            'avg_1d',
                            'tx_7d',
                            'avg_7d',
                            'tx_30d',
                            'avg_30d',
                            'time_btw_cc_tx'], axis=1)


In [4]:
# Create a new column distance
import math

df_payment.insert(5, "distance", 0)
for ind in df_payment.index:
    p1 = [0, 0]
    p2 = [df_payment['coord_x'][ind], df_payment['coord_y'][ind]]
    distance = math.sqrt(((p1[0] - p2[0]) ** 2) + ((p1[1] - p2[1]) ** 2))
    df_payment["distance"][ind] = distance

# extract hour
df_payment["ts_payment_at"] = pd.to_datetime(df_payment['payment_at'])

print("df_payment.dtype: ", df_payment.ts_payment_at.dtype)

df_payment["hour"] = df_payment.ts_payment_at.dt.hour
df_payment["dt_payment"] = df_payment.ts_payment_at.dt.date

/tmp/ipykernel_15846/1708626482.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_payment["distance"][ind] = distance
/tmp/ipykernel_15846/1708626482.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '342.64413025761877' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_payment["distance"][ind] = distance


df_payment.dtype:  datetime64[ns]


In [5]:
# 2nd filter dataframe
df_payment = df_payment.filter(['fraud',
                                'dt_payment',
                                'hour',
                                'distance',
                                'card_type',
                                'card_model',
                                'mcc',
                                'amount',
                                'tx_1d',
                                'avg_1d',
                                'tx_7d',
                                'avg_7d',
                                'tx_30d',
                                'avg_30d',
                                'time_btw_cc_tx'], axis=1)

In [6]:
df_payment = pd.get_dummies(df_payment,
                            columns=['card_model', ],
                            prefix='card_model',
                            dtype=int,
                            drop_first=False)

df_payment = pd.get_dummies(df_payment,
                            columns=['card_type', ],
                            prefix='card_type',
                            dtype=int,
                            drop_first=False)

In [7]:
df_payment.head(5)

,fraud,dt_payment,hour,distance,mcc,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx,card_model_CHIP,card_model_VIRTUAL,card_type_CREDIT
0,1,2024-03-19,21,342.644130,GAS,840.0,4,857.25,6,659.17,14,408.07,26,0,1,1
1,0,2024-04-18,18,50.990195,LAUNDRY,142.0,1,142.00,3,274.67,10,254.30,0,0,1,1
2,0,2024-03-23,15,89.185201,SPORTING,408.0,3,209.33,3,209.33,6,207.17,1802,1,0,1
3,0,2024-04-30,18,85.375641,CINEMA,106.0,2,109.50,2,109.50,8,210.75,997,1,0,1
4,0,2024-04-20,17,108.406642,LAUNDRY,106.0,2,303.00,7,551.00,13,743.23,0,1,0,1


# Feature Store

In [8]:
# using a dynamic name
#payment_feature_group_name = 'payment-feature-group-' + strftime('%d-%H-%M-%S', 
#                                                               gmtime())

payment_feature_group_name = 'payment-fraud-feature-group'

payment_feature_group = FeatureGroup(name=payment_feature_group_name,
                                     sagemaker_session=feature_store_session)

In [9]:
record_payment_name = "dt_payment"
event_time_feature_name = "EventTime"
current_time_sec = int(round(time.time()))

df_payment[event_time_feature_name] = pd.Series([current_time_sec]*len(df_payment),
                                                dtype="float64")
payment_feature_group.load_feature_definitions(data_frame=df_payment);

In [10]:
df_payment.head(3)

,fraud,dt_payment,hour,distance,mcc,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx,card_model_CHIP,card_model_VIRTUAL,card_type_CREDIT,EventTime
0,1,2024-03-19,21,342.644130,GAS,840.0,4,857.25,6,659.17,14,408.07,26,0,1,1,1.714865e+09
1,0,2024-04-18,18,50.990195,LAUNDRY,142.0,1,142.00,3,274.67,10,254.30,0,0,1,1,1.714865e+09
2,0,2024-03-23,15,89.185201,SPORTING,408.0,3,209.33,3,209.33,6,207.17,1802,1,0,1,1.714865e+09


In [11]:
# create a FeatureGroup

payment_feature_group.create(
    s3_uri=data_location,
    record_identifier_name=record_payment_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:908671954593:feature-group/payment-fraud-feature-group',
 'ResponseMetadata': {'RequestId': '2b11f5b3-b2ab-401d-82ec-bb54e613aa43',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2b11f5b3-b2ab-401d-82ec-bb54e613aa43',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '104',
   'date': 'Sat, 04 May 2024 23:30:32 GMT'},
  'RetryAttempts': 0}}

In [12]:
status = payment_feature_group.describe().get("FeatureGroupStatus")
print("payment_feature_group status: ", status)

payment_feature_group.describe()

payment_feature_group status:  Creating


{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:908671954593:feature-group/payment-fraud-feature-group',
 'FeatureGroupName': 'payment-fraud-feature-group',
 'RecordIdentifierFeatureName': 'dt_payment',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'fraud', 'FeatureType': 'Integral'},
  {'FeatureName': 'dt_payment', 'FeatureType': 'String'},
  {'FeatureName': 'hour', 'FeatureType': 'Integral'},
  {'FeatureName': 'distance', 'FeatureType': 'Fractional'},
  {'FeatureName': 'mcc', 'FeatureType': 'String'},
  {'FeatureName': 'amount', 'FeatureType': 'Fractional'},
  {'FeatureName': 'tx_1d', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_1d', 'FeatureType': 'Fractional'},
  {'FeatureName': 'tx_7d', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_7d', 'FeatureType': 'Fractional'},
  {'FeatureName': 'tx_30d', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_30d', 'FeatureType': 'Fractional'},
  {'FeatureName': 'time_btw_cc_tx', 'FeatureType': 'Integra

In [14]:
payment_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:908671954593:feature-group/payment-fraud-feature-group',
 'FeatureGroupName': 'payment-fraud-feature-group',
 'RecordIdentifierFeatureName': 'dt_payment',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'fraud', 'FeatureType': 'Integral'},
  {'FeatureName': 'dt_payment', 'FeatureType': 'String'},
  {'FeatureName': 'hour', 'FeatureType': 'Integral'},
  {'FeatureName': 'distance', 'FeatureType': 'Fractional'},
  {'FeatureName': 'mcc', 'FeatureType': 'String'},
  {'FeatureName': 'amount', 'FeatureType': 'Fractional'},
  {'FeatureName': 'tx_1d', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_1d', 'FeatureType': 'Fractional'},
  {'FeatureName': 'tx_7d', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_7d', 'FeatureType': 'Fractional'},
  {'FeatureName': 'tx_30d', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_30d', 'FeatureType': 'Fractional'},
  {'FeatureName': 'time_btw_cc_tx', 'FeatureType': 'Integra

# Put records in a feature group

In [15]:
# ingest data

payment_feature_group.ingest(
    data_frame=df_payment,
    max_workers=3,
    wait=True
)

IngestionManagerPandas(feature_group_name='payment-fraud-feature-group', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f3347d58160>, sagemaker_session=<sagemaker.session.Session object at 0x7f33486f7ee0>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f33462279d0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

# Get records from a feature group

In [ ]:
record_identifier_values = ["2024-03-19",
                            "2024-03-20"]

featurestore_runtime.batch_get_record(Identifiers=[{"FeatureGroupName": payment_feature_group_name,
                                                    "RecordIdentifiersValueAsString": record_identifier_values}])